# 使用说明
##### 此为伍仁杰(废物)编写的测试版脚本，不保证一定好用
1. 每天值班的小伙伴先登录国开行系统，贷款处理回执录入，不加查询条件，查询后点击左下角导出回执列表，筛选回执已录入的学生，再在待录入信息表格中将其去掉
1. 点击全部运行即可

In [1]:
import pandas as pd
import pyautogui as pag
import pyperclip
import time
import easygui
import numpy as np

In [2]:
file = easygui.fileopenbox("打开待录入文件")
data = pd.read_excel(file)

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:
def autoInput(df):
    name = df['姓名'].rstrip() # 去除空格
    sid = df['学号']
    # amount = df['贷款金额']
    school = df['院系']
    code = df['回执校验码']
    current_numid = df['序号']
    price = df['欠费']
    print("正在录入:{} {}".format(current_numid, name))

    pyperclip.copy(name)
    pag.click(x=300, y=230, duration=1, clicks=3) #三击姓名框
    pag.hotkey("ctrl","v")#粘贴名字
    pyperclip.copy(school)
    pag.click(x=300, y=200, duration=0.3, clicks=3) #三击院系框
    pag.hotkey("ctrl","v")
    pag.hotkey("alt","c")#查询

    # 检测是否有只有单个查询结果
    try:
        time.sleep(3.5)
        im = pag.screenshot(region=(50, 300, 250 ,50))
        # im.show()
        im = np.array(im)
        flag = 0
        for i in range(im.shape[1]):
            if (im[10][i][0]<200 and im[10][i][1]<200 and im[10][i][2]<200): # 找第一行像素是否有黑色
                flag = 1
                break
        if flag == 0:
            raise Exception("{} {}:没有找到该学生个人信息，请手动录入。".format(current_numid, name))
        for i in range(im.shape[1]):
            if (im[35][i][0]<200 and im[35][i][1]<200 and im[35][i][2]<200): # 找第二行像素是否有黑色
                raise Exception("{} {}:该学生出现重名情况，请手动录入。".format(current_numid, name))
    except Exception as e:
        raise

    pag.click(x=100, y=310, duration=0.5)#点录入回执
    # pag.click(x=800, y=430, duration=0.3, clicks=3) #三击院系
    # pag.hotkey("ctrl","v")
    pyperclip.copy(sid)
    pag.click(x=800,y=400, duration=0.5, clicks=2)#双击学号
    pag.hotkey("ctrl","v")
    pag.click(x=1000, y=600, duration=0.3) #单击高校账户选择
    pag.click(x=1000,y=590, duration=0.3) #选账户
    pyperclip.copy(price)
    pag.click(x=1160, y=580, duration=0.5, clicks=2) #双击欠费栏
    pag.hotkey("ctrl", "v") #粘贴欠费
    pyperclip.copy(code)
    pag.click(x=800, y=780, duration=0.3) #单击高校校验码
    pag.hotkey("ctrl","v")#粘贴校验码
    pag.click(x=600,y=890,duration=0.5)#点确定

    # 检测回执校验码是否正确
    try:
        time.sleep(1)
        im = pag.screenshot(region=(870, 470, 200 ,160))
        # im.show()
        im = np.array(im)
        for i in range(im.shape[1]):
            if (im[70][i][0]>250 and im[70][i][1]<230 and im[70][i][2]<10): #找像素是否那个警告标示的黄色
                raise Exception("{} {}:出现警告标示,判断为意外状况,可能是回执校验码错误,请手动录入。".format(current_numid, name))
    except Exception as e:
        raise

    # 检测欠款金额是否为0
    try:
        im = pag.screenshot(region=(830, 460, 280 ,170))
        # im.show()
        im = np.array(im)
        for i in range(im.shape[1]):
            if (im[70][i][0]<10 and abs(im[70][i][1]-110)<10 and abs(im[70][i][2]-200)<10): #找像素是否那个警告标示的蓝色
                pag.click(x=1040,y=610,duration=0.1)#点取消
                raise Exception("{} {}:出现警告标示,判断为意外状况,可能是欠款费用为0,请手动录入。".format(current_numid, name))
    except Exception as e:
        raise

    time.sleep(2.5)
    

In [ ]:
# 请修改start的值，使得程序从第start个学生开始录入
start = 788

pag.PAUSE = 0.3 #为pyautogui所有函数增加延迟，不需要多个sleep
text = "国开行录入脚本即将开始运行，请切换到国开行系统网页。\n点击该窗口的\"确定\"则3秒后开始运行,点击\"取消\"则不运行。\n程序运行中途如果想要强制停止,请把鼠标指针移至屏幕左上角。"
if(pag.confirm(text=text, title='运行提醒', buttons=['确定', '取消'])):
    try:
        time.sleep(3)
        data[start:].apply(autoInput, axis=1)
    except pag.FailSafeException as e:
        pag.alert(text="检测到鼠标指针移动至屏幕左上角，程序已停止运行。正在录入的学生序号请在代码的输出面板查看。", title='正常情况', button='OK')
    except Exception as e:
        pag.alert(text=e, title='异常情况', button='OK')
        print(e)
        exit(1)


正在录入:780 陈俊澎
正在录入:781 曹伟
正在录入:782 曾培伟
正在录入:783 种奕妃
正在录入:784 徐梦宏
正在录入:785 曹晓虹
正在录入:786 张少岭
正在录入:787 仝晨
787 仝晨:没有找到该学生个人信息，请手动录入。
